In [5]:
import smtplib
import imaplib
import email
from email.mime.text import MIMEText

# === BƯỚC 1: ĐỌC THÔNG TIN TÀI KHOẢN TỪ FILE ===
with open('email_credentials.txt', 'r', encoding='utf-8') as file:
    account_info = file.readlines()
    sender_email = account_info[0].strip()
    app_password = account_info[1].strip()

# === BƯỚC 2: ĐỌC THÔNG TIN EMAIL VÀ TIÊU CHÍ LỌC ===
with open('email_content.txt', 'r', encoding='utf-8') as file:
    email_content = file.read()

with open('recipient.txt', 'r', encoding='utf-8') as file:
    recipient_email = file.read().strip()

with open('email_filter.txt', 'r', encoding='utf-8') as file:
    filter_sender = file.read().strip()

# === BƯỚC 3: GỬI EMAIL ===
try:
    msg = MIMEText(email_content)
    msg['Subject'] = 'Email Tu Dong'
    msg['From'] = sender_email
    msg['To'] = recipient_email

    with smtplib.SMTP('smtp.gmail.com', 587) as smtp_server:
        smtp_server.starttls()
        smtp_server.login(sender_email, app_password)
        smtp_server.send_message(msg)
        print("✅ Email đã được gửi thành công.")
except Exception as e:
    print("❌ Lỗi khi gửi email:", e)

# === BƯỚC 4: NHẬN EMAIL ===
try:
    with imaplib.IMAP4_SSL('imap.gmail.com') as imap_server:
        imap_server.login(sender_email, app_password)
        imap_server.select('INBOX')

        # Tìm email từ người gửi cụ thể
        result, data = imap_server.search(None, f'FROM "{filter_sender}"')
        if result == 'OK' and data[0]:
            for num in data[0].split():
                result, msg_data = imap_server.fetch(num, '(RFC822)')
                email_msg = email.message_from_bytes(msg_data[0][1])
                print("\n📩 Tiêu đề:", email_msg["Subject"])
                if email_msg.is_multipart():
                    for part in email_msg.walk():
                        if part.get_content_type() == 'text/plain':
                            print("📨 Nội dung:", part.get_payload(decode=True).decode())
                            break
                else:
                    print("📨 Nội dung:", email_msg.get_payload(decode=True).decode())
                break  # Chỉ lấy 1 email đầu tiên
        else:
            print("📭 Không tìm thấy email phù hợp.")
except Exception as e:
    print("❌ Lỗi khi đọc email:", e)


✅ Email đã được gửi thành công.

📩 Tiêu đề: =?utf-8?b?RW1haWwgVOG7sSDEkOG7mW5nIHThu6sgVlNDb2Rl?=
📨 Nội dung: Chào bạn,

Tôi là Hứa Thị Thanh Hiền đây !

Cảm ơn bạn đã đọc!

Trân trọng
[Hiền]
